<a href="https://colab.research.google.com/github/dsri45/GreenlightHCP/blob/master/mlp_demo_neuromorphic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP Demo (Worked Example)

We will:
1. Create a simple dataset (two interleaving moons)
2. Train a small MLP in PyTorch
3. Evaluate accuracy
4. Visualize the decision boundary

---


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# Generate a toy dataset (2D points + binary labels)
X, y = make_moons(n_samples=2000, noise=0.25, random_state=42)

# Standardize features (helps training)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.long)

train_ds = torch.utils.data.TensorDataset(X_train_t, y_train_t)
val_ds   = torch.utils.data.TensorDataset(X_val_t, y_val_t)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=256, shuffle=False)

print("Train size:", len(train_ds), "Val size:", len(val_ds))

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim),
        )

    def forward(self, x):
        return self.net(x)

model = MLP(in_dim=2, hidden_dim=64, out_dim=2).to(device)
print(model)

In [ ]:
@torch.no_grad()
def accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == yb).sum().item()
        total += yb.numel()
    return correct / total

def train(model, train_loader, val_loader, epochs=60, lr=1e-2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    history = {"train_acc": [], "val_acc": [], "loss": []}

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * xb.size(0)

        avg_loss = running_loss / len(train_loader.dataset)
        tr_acc = accuracy(model, train_loader)
        va_acc = accuracy(model, val_loader)

        history["loss"].append(avg_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(va_acc)

        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch:03d} | loss {avg_loss:.4f} | train_acc {tr_acc:.3f} | val_acc {va_acc:.3f}")

    return history

history = train(model, train_loader, val_loader, epochs=60, lr=1e-2)

In [ ]:
plt.figure()
plt.plot(history["loss"])
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.figure()
plt.plot(history["train_acc"], label="train")
plt.plot(history["val_acc"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

print("Final val accuracy:", accuracy(model, val_loader))

In [ ]:
@torch.no_grad()
def plot_decision_boundary(model, X, y, title="Decision boundary"):
    model.eval()

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 300),
        np.linspace(y_min, y_max, 300)
    )
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_t = torch.tensor(grid, dtype=torch.float32).to(device)

    logits = model(grid_t)
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    Z = preds.reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=10)
    plt.title(title)
    plt.show()

plot_decision_boundary(model, X_val, y_val, title="MLP decision boundary (validation set)")

# MLP Demo (Fill-in-the-Blank)

Complete the TODOs to train an MLP classifier on the same moons dataset.

**Tasks**:
1. Build the model
2. Implement the forward pass
3. Implement the training step
4. Verify you reach good validation accuracy

---


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
X, y = make_moons(n_samples=2000, noise=0.25, random_state=123)
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=123, stratify=y
)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.long)

train_ds = torch.utils.data.TensorDataset(X_train_t, y_train_t)
val_ds   = torch.utils.data.TensorDataset(X_val_t, y_val_t)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=256, shuffle=False)

print("Train size:", len(train_ds), "Val size:", len(val_ds))

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        # TODO(1): Define 3 Linear layers and 2 ReLU activations.
        # Hint: the architecture is: Linear -> ReLU -> Linear -> ReLU -> Linear
        self.fc1 = None  # TODO: nn.Linear(in_dim, hidden_dim)
        self.fc2 = None  # TODO: nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = None  # TODO: nn.Linear(hidden_dim, out_dim)
        self.relu = None # TODO: nn.ReLU()

    def forward(self, x):
        # TODO(2): Implement the forward pass with ReLU between layers.
        # Return logits (no softmax needed; CrossEntropyLoss expects logits).
        pass

model = MLP(in_dim=2, hidden_dim=64, out_dim=2).to(device)
print(model)

In [ ]:
@torch.no_grad()
def accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == yb).sum().item()
        total += yb.numel()
    return correct / total

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

history = {"loss": [], "train_acc": [], "val_acc": []}

epochs = 60
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        # TODO(3): Standard training step:
        # - zero gradients
        # - forward pass to get logits
        # - compute loss
        # - backprop
        # - optimizer step
        optimizer.zero_grad()
        logits = ...          # TODO
        loss = ...            # TODO
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)

    avg_loss = running_loss / len(train_loader.dataset)
    tr_acc = accuracy(model, train_loader)
    va_acc = accuracy(model, val_loader)

    history["loss"].append(avg_loss)
    history["train_acc"].append(tr_acc)
    history["val_acc"].append(va_acc)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | loss {avg_loss:.4f} | train_acc {tr_acc:.3f} | val_acc {va_acc:.3f}")

In [ ]:
plt.figure()
plt.plot(history["loss"])
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.figure()
plt.plot(history["train_acc"], label="train")
plt.plot(history["val_acc"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

final_val = accuracy(model, val_loader)
print("Final validation accuracy:", final_val)

# Simple target threshold to confirm it worked (tweak if needed)
assert final_val > 0.85, "Try again: you should be able to reach > 0.85 val accuracy."
print("Looks good!")

In [ ]:
@torch.no_grad()
def plot_decision_boundary(model, X, y, title="Decision boundary"):
    model.eval()
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 300),
        np.linspace(y_min, y_max, 300)
    )
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_t = torch.tensor(grid, dtype=torch.float32).to(device)

    logits = model(grid_t)
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    Z = preds.reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=10)
    plt.title(title)
    plt.show()

plot_decision_boundary(model, X_val, y_val, title="Your MLP decision boundary (validation set)")

In [ ]:
# ML Start

In [ ]:
# =========================
# ML SECTION (Classical Baselines)
# =========================

import time
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def evaluate_predictions(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", zero_division=0
    )
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# Assumes these already exist from your preprocessing section:
# X_train_s, X_val_s, X_test_s, y_train, y_val, y_test

ml_models = {
    "Logistic Regression": LogisticRegression(max_iter=3000, random_state=SEED),
    "SVM (RBF)": SVC(kernel="rbf", C=1.0, gamma="scale", random_state=SEED),
    "Random Forest": RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_split=2, random_state=SEED
    ),
}

results = []

for name, model in ml_models.items():
    start = time.time()
    model.fit(X_train_s, y_train)
    train_time = time.time() - start

    val_pred = model.predict(X_val_s)
    test_pred = model.predict(X_test_s)

    val_metrics = evaluate_predictions(y_val, val_pred)
    test_metrics = evaluate_predictions(y_test, test_pred)

    row = {
        "model": name,
        "train_time_sec": round(train_time, 3),
        "val_accuracy": round(val_metrics["accuracy"], 4),
        "val_f1": round(val_metrics["f1"], 4),
        "test_accuracy": round(test_metrics["accuracy"], 4),
        "test_precision": round(test_metrics["precision"], 4),
        "test_recall": round(test_metrics["recall"], 4),
        "test_f1": round(test_metrics["f1"], 4),
    }
    results.append(row)

ml_results_df = pd.DataFrame(results).sort_values(by="test_f1", ascending=False).reset_index(drop=True)
ml_results_df


In [ ]:
# test_metrics_mlp should come from your DL section

best_ml_f1 = ml_results_df.loc[0, "test_f1"]
best_ml_acc = ml_results_df.loc[0, "test_accuracy"]
best_ml_name = ml_results_df.loc[0, "model"]

comparison_ml_vs_dl = pd.DataFrame([
    {"model": best_ml_name, "test_accuracy": best_ml_acc, "test_f1": best_ml_f1},
    {"model": "MLP (DL)", "test_accuracy": round(test_metrics_mlp["accuracy"], 4), "test_f1": round(test_metrics_mlp["f1"], 4)},
])

comparison_ml_vs_dl
